<a href="https://colab.research.google.com/github/yoonyerin/find_my_lover/blob/master/crop_head.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Pillow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import dlib
from PIL import Image


# 1. Data and trained_model(dlib)

In [ ]:
detector_HOG=dlib.get_frontal_face_detector()

model_path = "/content/drive/MyDrive/Find my lover/shape_predictor_68_face_landmarks.dat"
landmark_predictor = dlib.shape_predictor(model_path)

men_folder="/content/drive/MyDrive/Find my lover/men/"

for i in range(50):
  image_path = str(i+1)+".jpg"
  image=Image.open(men_folder+image_path).convert("RGB")
  image.save(str(i+1)+".png", "png")
  image_path=image_path.split(".")[0]+".png"

  img_cv = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)		# openCV로 이미지를 불러온기


# 2. Image Data Print

In [ ]:

img_list=[]
for i in range(50):
  image_path=str(i+1)+".png"
  img_list.append(cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB))

fig, axs=plt.subplots(10, 5, figsize=(20, 20))
for i in range(10):
  for j in range(5):
    axs[i, j].imshow(img_list[i*5+j])

plt.show()

In [ ]:
safe_img_list=[]
cropping_face=[]

# 3. crop head

In [ ]:
for i in range(50):
  try:
    img=img_list[i]
    men_dlib_rects=detector_HOG(img, 1)
    try: 
      l=men_dlib_rects[0].left()
      t = men_dlib_rects[0].top()
      r = men_dlib_rects[0].right()
      b = men_dlib_rects[0].bottom()
    except IndexError:
      print("사람이 아닙니당")

    points = landmark_predictor(img, men_dlib_rects[0])
      # face landmark 좌표를 저장
    list_points = list(map(lambda p: (p.x, p.y), points.parts()))
    
    point_hair_y=int(list_points[30][1]-(list_points[8][1]-list_points[30][1])*4/3)

    img_rgba=cv2.cvtColor(img, cv2.COLOR_RGB2RGBA)
    instance_img=img_rgba.copy()

    #non_trans.append(instance_img[0][0])

    img_skin=img.copy()

    img_skin=cv2.cvtColor(img_skin, cv2.COLOR_RGB2YCrCb)

    #하한

    lower = np.array([0,133,77], dtype = np.uint8)
    #상한
    upper = np.array([255,173,127], dtype = np.uint8)
    skin_mask=cv2.inRange(img_skin, lower, upper)

    skin_index=np.where(skin_mask==255)

    drop_list_points=[]
    for i in range(8): #8이 중간!  =>

      x_1, y_1= list_points[i]
      x_2, y_2=list_points[i+1]
      count=0
      for j in range(x_1, x_2):
        drop_list_points.append([j, int(y_1+(y_2-y_1)*count/(x_2-x_1))])
        count+=1 
    for i in range(8, 16): #8이 중간!  =>
      x_1, y_1= list_points[i]
      x_2, y_2=list_points[i+1]
      count=0
      for j in range(x_1, x_2):
        drop_list_points.append([j, int(y_1+(y_2-y_1)*count/(x_2-x_1))])
        count+=1 


    instance_img=img_rgba.copy()
    background_color=img_rgba[0][0]
    hair_colors=[]
    for i in range(drop_list_points[0][0], drop_list_points[-1][0]):
      hair_colors.append(instance_img[point_hair_y][i])

    for i in drop_list_points:
      instance_img[i[1]:,i[0]]=(0, 0, 0, 0)
    instance_img=img_rgba.copy()
    background_color=img_rgba[10][10]

    for i in drop_list_points:
      instance_img[i[1]:,i[0]]=(0, 0, 0, 0)
    for i in range(instance_img.shape[0]):
        for j in range(instance_img.shape[1]):
          if j>drop_list_points[-1][0] or j<drop_list_points[0][0]:
            flag=False
            if i<drop_list_points[-1][-1]:
              for hair_color in hair_colors:
                if all(instance_img[i][j] <= hair_color+[20, 20, 20, 0]) and all(instance_img[i][j]>=hair_color-[20, 20, 20, 0]):
                  flag=True
              if flag==True: continue
            instance_img[i][j]=(0, 0, 0, 0)
          elif i>t and i<b:
            continue
            

          if i>t and i<b and j>l and j<r and i in skin_index[0] and j in skin_index[1]:
              continue
          upper_bound=background_color+[25, 25, 25, 0]
          upper_bound[3]=255
          lower_bound=background_color-[25, 25, 25, 0]
          lower_bound[3]=0
          if all(instance_img[i][j] <= upper_bound) and all(instance_img[i][j]>=lower_bound):

              instance_img[i][j]=(0, 0, 0, 0)
    cropping_face.append(instance_img)
    #cropping_face.save("/content/drive/MyDrive/Find my lover/test/"+str(i)+".png")
    safe_img_list.append(img)
  except IndexError:
    print("index error jump")
    


In [ ]:
#for i in cropping_face:
#  i.save("/content/drive/MyDrive/Find my lover/test/"+str(i)+".png")

# 4. crop 결과 출력

In [ ]:
fig, axs=plt.subplots(len(safe_img_list), 2, figsize=(20, 200))
for i in range(len(safe_img_list)):
  axs[i, 0].imshow(safe_img_list[i])
  axs[i, 1].imshow(cropping_face[i])